In [1]:
import csv
import math

import matplotlib.pyplot as plt
import numpy as np

from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [2]:
# build the VGG16 network
input_tensor = Input(shape=(224,224,3))
model = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)

In [3]:
# build a classifier model to put on top of the convolutional model
x = model.output
x = Flatten(input_shape=(model.output_shape[1:]))(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dense(4, activation='softmax', name='output', kernel_initializer='glorot_uniform')(x)

# add new classifier model on top of convolutional base
new_model = Model(model.input, x)

In [4]:
# set the first 19 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in new_model.layers[:19]:
    layer.trainable = False

In [5]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [6]:
# Use SGD and Categorical CE Loss
#sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
#new_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# RMSprop
new_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

In [7]:
# read the CSV into memory
prices = []
image_paths = []

data_path = "../datasets/bikes_im/"
with open("bikes_classified.csv") as file:
    reader = csv.reader(file)
    i = -1
    for row in reader:
        i += 1
        index = row[0]
        name = row[1]
        msrp = row[2]
        label = row[3]
        
        image_path = data_path + index + '.jpg'
        image_paths.append(image_path)
        prices.append(str(label))

In [8]:
prices[-1]

'100'

In [9]:
def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 4)) # Change to one-hot
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 4)) # Change to one-hot
            
            for i, index in enumerate(batch_indices):
                img = image.load_img(image_paths[index], target_size=(224, 224))
                X[i, :, :, :] = image.img_to_array(img)
                # Convert to 1 hot vector
                p = prices[index]
                if p == "25":
                    Y[i,:] = np.array([1,0,0,0])
                if p == "50":
                    Y[i,:] = np.array([0,1,0,0])
                if p == "75":
                    Y[i,:] = np.array([0,0,1,0])
                if p == "100":
                    Y[i,:] = np.array([0,0,0,1])
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)

In [10]:
train_indices = np.load("bikes_train_indices.npy")
test_indices = np.load("bikes_test_indices.npy")
print(train_indices.shape)
print(test_indices.shape)

(19658,)
(2185,)


In [11]:
epochs = 7
minibatch_size = 64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=image_generator(test_indices, minibatch_size),
    nb_val_samples=test_steps)

Epoch 1/7


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  del sys.path[0]
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=308, epochs=7, validation_data=<generator..., validation_steps=35)`
  del sys.path[0]


308/308 [==============================] - 260s - loss: 8.1458 - acc: 0.4724 - val_loss: 6.7911 - val_acc: 0.5482
Epoch 2/7
308/308 [==============================] - 244s - loss: 6.7782 - acc: 0.5441 - val_loss: 6.7387 - val_acc: 0.5456
Epoch 3/7
308/308 [==============================] - 244s - loss: 6.0370 - acc: 0.5738 - val_loss: 5.3828 - val_acc: 0.5976
Epoch 4/7
308/308 [==============================] - 244s - loss: 4.2859 - acc: 0.6043 - val_loss: 1.0118 - val_acc: 0.6904
Epoch 5/7
308/308 [==============================] - 244s - loss: 0.9698 - acc: 0.6942 - val_loss: 0.7514 - val_acc: 0.7190
Epoch 6/7
308/308 [==============================] - 244s - loss: 0.7018 - acc: 0.7526 - val_loss: 0.7405 - val_acc: 0.7390
Epoch 7/7
308/308 [==============================] - 244s - loss: 0.6248 - acc: 0.7785 - val_loss: 0.7428 - val_acc: 0.7590


In [12]:
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=image_generator(test_indices, minibatch_size),
    nb_val_samples=test_steps)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=308, epochs=7, validation_data=<generator..., validation_steps=35)`
  


Epoch 1/7
308/308 [==============================] - 243s - loss: 0.5658 - acc: 0.8001 - val_loss: 0.6909 - val_acc: 0.7728
Epoch 2/7
308/308 [==============================] - 244s - loss: 0.5195 - acc: 0.8182 - val_loss: 0.6896 - val_acc: 0.7892
Epoch 3/7
308/308 [==============================] - 244s - loss: 0.4793 - acc: 0.8356 - val_loss: 0.7018 - val_acc: 0.7852
Epoch 4/7
308/308 [==============================] - 244s - loss: 0.4491 - acc: 0.8532 - val_loss: 0.7078 - val_acc: 0.7941
Epoch 5/7
308/308 [==============================] - 244s - loss: 0.4233 - acc: 0.8590 - val_loss: 0.6998 - val_acc: 0.8039
Epoch 6/7
308/308 [==============================] - 244s - loss: 0.3925 - acc: 0.8696 - val_loss: 0.7438 - val_acc: 0.7768
Epoch 7/7
308/308 [==============================] - 243s - loss: 0.3727 - acc: 0.8794 - val_loss: 0.7724 - val_acc: 0.8021


In [13]:
new_model.save('bikes_classification_best.hdf5')